In [ ]:
import os
import pandas as pd
# from datetime import date

directory = os.fsencode("./trading_data")

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".csv"):
        # import Stock data and convert date format
        df1 = pd.read_csv("./trading_data/" + filename)
        df1["Date"] = pd.to_datetime(df1["Date"], utc=True).dt.tz_convert(None).dt.date

        # import sentiment data and convert dateformat
        df2 = pd.read_csv("sentiment_avg.csv")
        df2["Publish Date"] = pd.to_datetime(df2["Publish Date"]).dt.date

        # merge dataframes
        df = pd.merge(df1, df2, left_on="Date", right_on="Publish Date")
        df = df.drop("Publish Date", axis=1)
        df.rename(columns={"Average": "Sentiment Average"}, inplace=True)

        # rewrite existing csv file with additional changes
        df.to_csv("./trading_data/" + filename, index=False)


In [1]:
import plotly.graph_objects as go

# Sample data
categories = ['A', 'B', 'C', 'D', 'E']
values1 = [79.8, 96.3, 71.5, 97.3, 90.9]  # First dataset
values2 = [72.2, 93.6, 75.7, 96.4, 91.8]  # Second dataset
values3 = [63.6, 90.6, 62.1, 90.2, 87.4]  # Third dataset

# Create figure
fig = go.Figure()

# Add first bar set (with hatch pattern)
fig.add_trace(go.Bar(
    x=categories,
    y=values1,
    name='Set 1',
    marker=dict(color='royalblue', pattern_shape="x")
))

# Add second bar set
fig.add_trace(go.Bar(
    x=categories,
    y=values2,
    name='Set 2',
    marker=dict(color='lightblue')
))

# Add third bar set
fig.add_trace(go.Bar(
    x=categories,
    y=values3,
    name='Set 3',
    marker=dict(color='gray')
))

# Layout settings
fig.update_layout(
    barmode='group',  # Grouped bars
    title='Comparison Bar Chart with Hatch Patterns',
    xaxis_title='Categories',
    yaxis_title='Values',
    template='plotly_white'
)

# Show the figure
fig.show()


In [ ]:
import plotly.graph_objects as go

categories =  ["PPO", "RecurrentPPO", "CLSTM-PPO", "Ensemble", "DJI"]